In [1]:
import sys
sys.argv = ['main.py']

import options

import torch
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
torch.multiprocessing.set_start_method('spawn', force=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import torchvision
from torchvision import datasets, transforms

import random
import numpy as np
import os

from models.ResNet import ResNet18

In [2]:
import argparse
parser = argparse.ArgumentParser()

parser.add_argument("--net", default='ResNet18', type=str, help="Net for poison generation.")
parser.add_argument("--clean_target_id", default=886, type=int, help="Index of the target, selected from the testset")
parser.add_argument("--clean_label", default=torch.tensor([0,]).to(device), help="Class index of the target")
parser.add_argument("--root", default='../datasets/', help="Path of the dataset.")
parser.add_argument("--batch_size", default=512, type=int)
parser.add_argument("--num_poison", default=50, type=int, help="Number of the poison images")
parser.add_argument("--alpha", default=0.1, type=float, help="Ratio of target image component.")
parser.add_argument("--beta", default=0.75, type=float)
parser.add_argument("--eps", default=0.03137254901, type=float, help="Clip Value.")
parser.add_argument("--step_size", default=0.00784313725, type=float, help="LR for the inner optimizer.")
parser.add_argument("--lr", default=1e-3, type=float, help="LR for the outter optimizer.")
parser.add_argument("--ZO", action="store_true", help="use Zeroth-Order method to get grad.")

# args = parser.parse_args(args=["--ZO"])
args = parser.parse_args(args=[])
print(args)

Namespace(net='ResNet18', clean_target_id=886, clean_label=tensor([0], device='cuda:0'), root='../datasets/', batch_size=512, num_poison=50, alpha=0.1, beta=0.75, eps=0.03137254901, step_size=0.00784313725, lr=0.001, ZO=False)


In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

train_transform = [
    transforms.ToTensor()
]
test_transform = [
    transforms.ToTensor()
]
train_transform = transforms.Compose(train_transform)
test_transform = transforms.Compose(test_transform)

clean_train_dataset = datasets.CIFAR10(root='../datasets', train=True, download=True, transform=train_transform)
clean_test_dataset = datasets.CIFAR10(root='../datasets', train=False, download=True, transform=test_transform)

clean_train_loader = DataLoader(dataset=clean_train_dataset, batch_size=args.batch_size,
                                shuffle=False, pin_memory=True,
                                drop_last=False, num_workers=0)
clean_test_loader = DataLoader(dataset=clean_test_dataset, batch_size=args.batch_size,
                                shuffle=False, pin_memory=True,
                                drop_last=False, num_workers=0)

clean_target = torch.tensor(clean_test_dataset.data[args.clean_target_id].astype(np.float32)/255).to(device)

_CLASS_ = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import matplotlib.pyplot as plt

def npshow(img, trans=(0, 1, 2)):  # (32, 32, 3)
    fig = plt.figure(figsize=(8, 8), dpi=40, facecolor='w', edgecolor='k')
    transimg = np.transpose(img, trans)
    plt.imshow(transimg)
    plt.show()

In [5]:
base_model = ResNet18()
base_model = base_model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=base_model.parameters(), lr=0.1, weight_decay=0.0005, momentum=0.9)

In [6]:
from tools import find_similar_img, linear_interpolation 
from poisoned_datasets import UnlearnableDataset, create_poison_data

similar_imgs = find_similar_img(clean_target, orig_label = int(args.clean_label), k = args.num_poison)
target_train_dataset = linear_interpolation(clean_target, similar_imgs, alpha=args.alpha)
raw_dataset = create_poison_data(target_train_dataset) # float, save into .pt
unlearnable_train_dataset = UnlearnableDataset(args.clean_label)
perturb_dataset = ConcatDataset([clean_train_dataset, unlearnable_train_dataset])
purturb_loader = DataLoader(dataset=perturb_dataset, batch_size=args.batch_size,
                                shuffle=False, pin_memory=False,
                                drop_last=False, num_workers=0)

Files already downloaded and verified


100%|██████████| 50000/50000 [00:08<00:00, 5663.62step/s]


Interpolating images, Alpha 0.1


100%|██████████| 50000/50000 [00:07<00:00, 6589.64step/s]
/home/mist/cloud/ZOUL/tools.py:54: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /home/mist/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  ip_img = torch.tensor([img.cpu().detach().numpy() for img in ip_img]).to(device)


In [7]:
from Pertubation import PerturbationTool
from trainers import generate_noise

noise_generator = PerturbationTool(epsilon=args.eps, num_steps=20, step_size=args.step_size)
perturb_img, noise = generate_noise(base_model, criterion, optimizer, noise_generator, purturb_loader, ZO_method=args.ZO, max_iter=10)

  9%|▉         | 9/98 [00:23<03:55,  2.65s/it]


KeyboardInterrupt: 

In [ ]:
import numpy as np

uni_train_transform = [
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
]
uni_train_transform = transforms.Compose(uni_train_transform)
perturb_untrained_dataset_image = []
perturb_untrained_dataset_label = []
poisoned_train_dataset = []

for single_data in perturb_dataset:
    perturb_untrained_dataset_image.append(single_data[0])
    perturb_untrained_dataset_label.append(single_data[1])

for index, single_data in enumerate(perturb_untrained_dataset_image):
    if index > 49999:
        single_data = single_data + noise[index]
    poisoned_data = single_data.permute([1, 2, 0]).detach().to('cpu').clamp_(0, 1).numpy().astype(np.float32)
    poisoned_train_dataset.append((poisoned_data, perturb_untrained_dataset_label[index]))
    

In [ ]:
import matplotlib.pyplot as plt

def npshow(img, trans=(0, 1, 2)):  # (32, 32, 3)
    fig = plt.figure(figsize=(8, 8), dpi=40, facecolor='w', edgecolor='k')
    transimg = np.transpose(img, trans)
    plt.imshow(transimg)
    plt.show()

def get_noise_norm(idx):
    idx += 50000
    x = noise[idx]
    x_min = torch.min(x)
    x_max = torch.max(x)
    noise_norm = (x - x_min) / (x_max - x_min)
    noise_norm = torch.clamp(noise_norm, 0, 1)
    return noise_norm

# bias = random.randint(0, args.num_poison - 1)
bias = 0

print("bias:", bias)
print("similar_imgs:", similar_imgs)
print("Clear target:")
npshow(clean_target.detach().to('cpu').numpy())

print("UNINTERPOLATION_SIM_IMAGE Poisoned_train_dataset:")
npshow(poisoned_train_dataset[similar_imgs[bias]][0])

print("NOISED_POISON Unlearnable_train_dataset:")
npshow(perturb_dataset[50000+bias][0].detach().to('cpu').numpy(), [1, 2, 0])

noise_n = get_noise_norm(bias)
print("noise_norm:")
npshow(noise_n.detach().to('cpu').numpy(), [1, 2, 0])

In [ ]:
from tqdm import tqdm

model = ResNet50()
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1, weight_decay=0.0005, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=0)

poisoned_loader = DataLoader(dataset=poisoned_train_dataset, batch_size=args.batch_size,
                                shuffle=False, pin_memory=False,
                                drop_last=False, num_workers=0)

epoch = 0
condition = True
while condition:
    # Train
    model.train(True)
    pbar = tqdm(poisoned_loader, total=len(poisoned_loader))
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        images = images.permute([0, 3, 1, 2])
        model.zero_grad()
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        
        _, predicted = torch.max(logits.data, 1)
        acc = (predicted == labels).sum().item()/labels.size(0)
        pbar.set_description("Acc %.2f Loss: %.2f" % (acc*100, loss))
    scheduler.step()
    # optimizer.step()

    # Eval
    model.eval()
    model.train(False)
    correct, total = 0, 0
    clear_label_correct = 0
    for i, (images, labels) in enumerate(clean_test_loader):
        images, labels = images.to(device), labels.to(device)
        with torch.no_grad():
            logits = model(images)
            _, predicted = torch.max(logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = correct / total
    # tqdm.write("Correct: %d, Total: %d" % (correct, total))
    tqdm.write('Epoch %d Clean Accuracy %.2f\n' % (epoch, acc*100))
    epoch += 1
    
    if acc > args.beta or epoch > 30:
        condition = False


In [ ]:
test_image = clean_target.permute([2, 0, 1]).unsqueeze(0)
print(test_image.shape)
logits = model(test_image)
_, pred = torch.max(logits, 1)
print("Original:", _CLASS_[int(args.clean_label)], "Prediction:", _CLASS_[int(pred)])
print(logits)

In [ ]:
logits = logits[0].detach().cpu().numpy()
fig, ax = plt.subplots(figsize=(10, 8))
ax.bar(x=_CLASS_, height=logits, width=0.8, align="center")
xticks = ax.get_xticks()
for i in range(len(logits)):
    xy = (xticks[i], logits[i])